# Probability Density
* `pdf1` -- Density estimatation with histogram

Transform ratio/interval-scale data when the underlying (obscured) real dataset is actually categorical.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# add path
import sys; import os; sys.path.append(os.path.realpath("../"))

In [3]:
# demo datasets
from datasets.demo1 import X_train, Y_train, fold_ids, X_valid, Y_valid, meta as meta_data
#meta_data

In [4]:
# transformer implementations
typ = 'pdf1'

if typ is 'pdf1':
    from verto.pdf1 import trans, meta
    trans.set_params(**{'bins': 16})
else:
    tmp = __import__("verto."+typ, fromlist=['trans', 'meta'])
    trans = tmp.trans
    meta = tmp.meta

In [5]:
meta

{'id': 'pdf1',
 'name': 'Prob Density',
 'description': 'Probability density estimated by a histogram. Lookup density based on fitted bin edges',
 'keywords': ['histogram', 'probability density'],
 'feature_names_prefix': 'pdf_hist'}

## Transform

In [6]:
%%time
trans.fit(X_train, Y_train)

CPU times: user 10.7 ms, sys: 2.98 ms, total: 13.7 ms
Wall time: 11.3 ms


HistDensity(bins=16)

In [7]:
%%time
X_new = trans.transform(X_train)

CPU times: user 62.9 ms, sys: 7.72 ms, total: 70.6 ms
Wall time: 66.7 ms


In [8]:
from seasalt import create_feature_names
feature_names = create_feature_names(meta['feature_names_prefix'], X_new.shape[1])
print(feature_names)

['pdf_hist_0', 'pdf_hist_1', 'pdf_hist_2', 'pdf_hist_3', 'pdf_hist_4', 'pdf_hist_5', 'pdf_hist_6', 'pdf_hist_7', 'pdf_hist_8', 'pdf_hist_9', 'pdf_hist_10', 'pdf_hist_11', 'pdf_hist_12', 'pdf_hist_13', 'pdf_hist_14', 'pdf_hist_15', 'pdf_hist_16', 'pdf_hist_17', 'pdf_hist_18', 'pdf_hist_19', 'pdf_hist_20', 'pdf_hist_21', 'pdf_hist_22', 'pdf_hist_23', 'pdf_hist_24', 'pdf_hist_25', 'pdf_hist_26', 'pdf_hist_27', 'pdf_hist_28', 'pdf_hist_29']


In [9]:
import pandas as pd
df_new = pd.DataFrame(data=X_new, columns=feature_names)

## Evaluate
- check if the PCA components are "good" predictors
- eyeball the p-values of the logistic regression coefficients

In [10]:
df_new.head()

,pdf_hist_0,pdf_hist_1,pdf_hist_2,pdf_hist_3,pdf_hist_4,pdf_hist_5,pdf_hist_6,pdf_hist_7,pdf_hist_8,pdf_hist_9,...,pdf_hist_20,pdf_hist_21,pdf_hist_22,pdf_hist_23,pdf_hist_24,pdf_hist_25,pdf_hist_26,pdf_hist_27,pdf_hist_28,pdf_hist_29
0,3.993490,2.572864,5.312774,5.205121,4.237231,5.744848,6.432161,2.572864,4.079769,2.572864,...,3.964799,3.859296,2.626229,2.597378,1.420375,8.804567,3.565670,1.358316,2.572864,9.976065
1,2.662327,3.859296,5.312774,2.602561,1.412410,4.308636,2.572864,2.572864,2.719846,3.859296,...,1.321600,3.859296,7.878686,6.493444,4.261125,5.869712,1.782835,2.716631,1.286432,6.650710
2,3.993490,3.859296,1.328193,5.205121,5.649642,1.436212,1.286432,1.286432,2.719846,2.572864,...,6.607998,2.572864,5.252457,7.792133,11.362999,2.934856,3.565670,4.074947,5.145729,6.650710
3,5.324654,5.145729,6.640967,6.506401,4.237231,1.436212,6.432161,10.291457,2.719846,9.005025,...,5.286398,3.859296,3.939343,6.493444,2.840750,1.467428,7.131340,6.791578,2.572864,6.650710
4,3.993490,1.286432,5.312774,10.410242,5.649642,1.436212,7.718593,5.145729,4.079769,1.286432,...,3.964799,1.286432,5.252457,5.194755,5.681500,8.804567,10.697010,2.716631,3.859296,6.650710


In [11]:
import statsmodels.api as sm
#lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit()
lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit_regularized(method='l1', alpha=.5)
print(lr.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.26910246683396216
            Iterations: 89
            Function evaluations: 94
            Gradient evaluations: 89
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  398
Model:                          Logit   Df Residuals:                      368
Method:                           MLE   Df Model:                           29
Date:                Fri, 05 Apr 2019   Pseudo R-squ.:                  0.6173
Time:                        15:36:10   Log-Likelihood:                -100.72
converged:                       True   LL-Null:                       -263.17
                                        LLR p-value:                 9.259e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
